In [1]:
import numpy as np
import collections.abc
collections.Iterable = collections.abc.Iterable
from collections import OrderedDict
import pylogit as pl
import pandas as pd
import scipy.special
import json

In [2]:
df = pd.read_csv('history.csv')
df

,round,arena,pirate1,pirate2,pirate3,pirate4,fa1,fa2,fa3,fa4,...,nfa4,opening_odds1,opening_odds2,opening_odds3,opening_odds4,closing_odds1,closing_odds2,closing_odds3,closing_odds4,winner
0,3574,0,7,19,4,13,0,0,0,0,...,0,5,2,3,9,6,2,3,11,3
1,3574,1,18,12,17,14,0,0,0,0,...,0,13,2,3,3,13,2,3,3,4
2,3574,2,2,16,9,3,0,0,0,0,...,0,13,2,5,4,13,2,6,5,2
3,3574,3,10,20,1,6,0,0,0,0,...,0,13,10,2,13,13,13,2,13,3
4,3574,4,8,15,11,5,0,0,0,0,...,0,13,2,11,6,13,2,13,6,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17450,8763,0,1,15,14,11,4,2,0,0,...,-2,3,2,13,13,4,2,13,13,2
17451,8763,1,9,3,8,20,-1,3,2,0,...,-1,13,7,6,2,13,9,9,2,2
17452,8763,2,2,12,6,19,1,0,1,-2,...,-3,10,8,4,2,13,10,5,2,4
17453,8763,3,10,7,13,18,0,3,0,-2,...,-3,13,2,3,9,13,2,3,11,2


In [3]:
df.describe()

,round,arena,pirate1,pirate2,pirate3,pirate4,fa1,fa2,fa3,fa4,...,nfa4,opening_odds1,opening_odds2,opening_odds3,opening_odds4,closing_odds1,closing_odds2,closing_odds3,closing_odds4,winner
count,17455.000000,17455.000000,17455.000000,17455.000000,17455.000000,17455.000000,17455.000000,17455.000000,17455.000000,17455.000000,...,17455.000000,17455.000000,17455.000000,17455.000000,17455.000000,17455.000000,17455.000000,17455.000000,17455.000000,17455.000000
mean,6895.217416,2.000000,10.594958,10.437353,10.527757,10.439931,0.970496,0.988026,0.969350,0.983157,...,-1.209625,7.924148,7.042051,5.981438,4.687310,8.248525,7.416786,6.361673,5.057405,2.747522
std,1124.586097,1.414254,5.748542,5.784008,5.782101,5.749531,1.732478,1.735760,1.733119,1.732663,...,0.970576,4.460331,4.421288,4.184334,3.634178,4.532492,4.556024,4.384231,3.926012,1.104911
min,3574.000000,0.000000,1.000000,1.000000,1.000000,1.000000,-5.000000,-5.000000,-5.000000,-5.000000,...,-6.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000
25%,5962.000000,1.000000,6.000000,5.000000,5.000000,5.000000,0.000000,0.000000,0.000000,0.000000,...,-2.000000,3.000000,3.000000,2.000000,2.000000,3.000000,3.000000,2.000000,2.000000,2.000000
50%,6918.000000,2.000000,11.000000,10.000000,11.000000,10.000000,1.000000,1.000000,1.000000,1.000000,...,-1.000000,7.000000,6.000000,4.000000,3.000000,9.000000,7.000000,5.000000,3.000000,3.000000
75%,7891.000000,3.000000,16.000000,15.000000,16.000000,15.000000,2.000000,2.000000,2.000000,2.000000,...,0.000000,13.000000,13.000000,10.000000,6.000000,13.000000,13.000000,11.000000,7.000000,4.000000
max,8763.000000,4.000000,20.000000,20.000000,20.000000,20.000000,8.000000,8.000000,8.000000,9.000000,...,0.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,4.000000


In [4]:
# combine columns pirate1 through pirate4 into a single column containing a list of pirate choices
def convert_df(df_orig):
    df = df_orig.copy()
    df['pirates'] = df[['pirate1', 'pirate2', 'pirate3', 'pirate4']].values.tolist()
    df['fas'] = df[['fa1', 'fa2', 'fa3', 'fa4']].values.tolist()
    df['pfas'] = df[['pfa1', 'pfa2', 'pfa3', 'pfa4']].values.tolist()
    df['nfas'] = df[['nfa1', 'nfa2', 'nfa3', 'nfa4']].values.tolist()
    df['opening_odds'] = df[['opening_odds1', 'opening_odds2', 'opening_odds3', 'opening_odds4']].values.tolist()
    df['closing_odds'] = df[['closing_odds1', 'closing_odds2', 'closing_odds3', 'closing_odds4']].values.tolist()
    df = df.drop(['pirate1', 'pirate2', 'pirate3', 'pirate4'], axis=1)
    df = df.drop(['fa1', 'fa2', 'fa3', 'fa4'], axis=1)
    df = df.drop(['pfa1', 'pfa2', 'pfa3', 'pfa4'], axis=1)
    df = df.drop(['nfa1', 'nfa2', 'nfa3', 'nfa4'], axis=1)
    df = df.drop(['opening_odds1', 'opening_odds2', 'opening_odds3', 'opening_odds4'], axis=1)
    df = df.drop(['closing_odds1', 'closing_odds2', 'closing_odds3', 'closing_odds4'], axis=1)
    df['match_id'] = df['round'] * 5 + df['arena']
    return df
df = convert_df(df)
df

,round,arena,winner,pirates,fas,pfas,nfas,opening_odds,closing_odds,match_id
0,3574,0,3,"[7, 19, 4, 13]","[0, 0, 0, 0]","[0, 0, 0, 0]","[0, 0, 0, 0]","[5, 2, 3, 9]","[6, 2, 3, 11]",17870
1,3574,1,4,"[18, 12, 17, 14]","[0, 0, 0, 0]","[0, 0, 0, 0]","[0, 0, 0, 0]","[13, 2, 3, 3]","[13, 2, 3, 3]",17871
2,3574,2,2,"[2, 16, 9, 3]","[0, 0, 0, 0]","[0, 0, 0, 0]","[0, 0, 0, 0]","[13, 2, 5, 4]","[13, 2, 6, 5]",17872
3,3574,3,3,"[10, 20, 1, 6]","[0, 0, 0, 0]","[0, 0, 0, 0]","[0, 0, 0, 0]","[13, 10, 2, 13]","[13, 13, 2, 13]",17873
4,3574,4,2,"[8, 15, 11, 5]","[0, 0, 0, 0]","[0, 0, 0, 0]","[0, 0, 0, 0]","[13, 2, 11, 6]","[13, 2, 13, 6]",17874
...,...,...,...,...,...,...,...,...,...,...
17450,8763,0,2,"[1, 15, 14, 11]","[4, 2, 0, 0]","[5, 3, 2, 2]","[-1, -1, -2, -2]","[3, 2, 13, 13]","[4, 2, 13, 13]",43815
17451,8763,1,2,"[9, 3, 8, 20]","[-1, 3, 2, 0]","[0, 4, 4, 1]","[-1, -1, -2, -1]","[13, 7, 6, 2]","[13, 9, 9, 2]",43816
17452,8763,2,4,"[2, 12, 6, 19]","[1, 0, 1, -2]","[3, 0, 2, 1]","[-2, 0, -1, -3]","[10, 8, 4, 2]","[13, 10, 5, 2]",43817
17453,8763,3,2,"[10, 7, 13, 18]","[0, 3, 0, -2]","[1, 3, 3, 1]","[-1, 0, -3, -3]","[13, 2, 3, 9]","[13, 2, 3, 11]",43818


In [5]:
# explode each row of df into four rows, so that the resulting df
# contains columns of round, arena, win (0 or 1), pirate, fa, opening_odds, closing_odds, match_id
def convert_to_long_format_with_win(df):
    long_format_columns = ['round', 'arena', 'win', 'pirate', 'fa', 'pfa', 'nfa', 'position', 'position_factor', 'is_pos1', 'is_pos2', 'is_pos3', 'is_pos4', 'opening_odds', 'closing_odds', 'match_id']
    # positional effect is significant but sorta non-linear; this is taken from a model data
    # and we'll calculate each pirates "suscetibility to positional effects" by fitting a multiplier
    position_factor = [-0.1821, 0, 0.2814, 0.5523]
    long_format = pd.DataFrame([
        (
            tup.round,
            tup.arena,
            1 if 'winner' in df.columns and tup.winner == (i + 1) else 0,
            tup.pirates[i],
            tup.fas[i],
            tup.pfas[i],
            tup.nfas[i],
            i + 1,
            position_factor[i],
            1 if i + 1 == 1 else 0,
            1 if i + 1 == 2 else 0,
            1 if i + 1 == 3 else 0,
            1 if i + 1 == 4 else 0,
            tup.opening_odds[i],
            tup.closing_odds[i],
            tup.match_id
        )
        for tup in df.itertuples() for i in range(4)
    ],columns=long_format_columns)
    long_format['log_opening_implied_winrate'] = np.log(1.0 / long_format['opening_odds'])
    return long_format

def convert_to_long_format_without_win(df):
    return convert_to_long_format_with_win(df).drop(['win'], axis=1)

long_format = convert_to_long_format_with_win(df)
long_format

,round,arena,win,pirate,fa,pfa,nfa,position,position_factor,is_pos1,is_pos2,is_pos3,is_pos4,opening_odds,closing_odds,match_id,log_opening_implied_winrate
0,3574,0,0,7,0,0,0,1,-0.1821,1,0,0,0,5,6,17870,-1.609438
1,3574,0,0,19,0,0,0,2,0.0000,0,1,0,0,2,2,17870,-0.693147
2,3574,0,1,4,0,0,0,3,0.2814,0,0,1,0,3,3,17870,-1.098612
3,3574,0,0,13,0,0,0,4,0.5523,0,0,0,1,9,11,17870,-2.197225
4,3574,1,0,18,0,0,0,1,-0.1821,1,0,0,0,13,13,17871,-2.564949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69815,8763,3,0,18,-2,1,-3,4,0.5523,0,0,0,1,9,11,43818,-2.197225
69816,8763,4,0,17,5,6,-1,1,-0.1821,1,0,0,0,2,2,43819,-0.693147
69817,8763,4,0,5,1,1,0,2,0.0000,0,1,0,0,5,5,43819,-1.609438
69818,8763,4,0,16,1,2,-1,3,0.2814,0,0,1,0,4,4,43819,-1.386294


In [6]:
PIRATE_NAMES = {
    1: "Dan",
    2: "Sproggie",
    3: "Orvinn",
    4: "Lucky",
    5: "Edmund",
    6: "Peg Leg",
    7: "Bonnie",
    8: "Puffo",
    9: "Stuff",
    10: "Squire",
    11: "Crossblades",
    12: "Stripey",
    13: "Ned",
    14: "Fairfax",
    15: "Gooblah",
    16: "Franchisco",
    17: "Federismo",
    18: "Blackbeard",
    19: "Buck",
    20: "Tailhook",
}

In [7]:
pirates_including_the_first = list(range(1, 21))
pirates_except_the_first = list(range(2, 21))

In [8]:
POSITIVE_FAS = {
    1: {1: 2, 2: 0, 3: 0, 4: 1, 5: 0, 6: 1, 7: 1, 8: 1, 9: 0, 10: 1, 11: 0, 12: 2, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 1, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 2, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    2: {1: 1, 2: 0, 3: 0, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 0, 10: 1, 11: 0, 12: 1, 13: 1, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 1, 34: 0, 35: 1, 36: 1, 37: 0, 38: 0, 39: 0, 40: 0},
    3: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 1, 15: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1, 22: 1, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 1, 29: 0, 30: 0, 31: 0, 32: 1, 33: 0, 34: 1, 35: 0, 36: 0, 37: 1, 38: 0, 39: 0, 40: 1},
    4: {1: 0, 2: 0, 3: 1, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 1, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 1, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 1, 26: 0, 27: 0, 28: 1, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 1, 40: 1},
    5: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 1, 19: 0, 20: 0, 21: 0, 22: 0, 23: 1, 24: 1, 25: 0, 26: 0, 27: 0, 28: 0, 29: 1, 30: 0, 31: 1, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 1, 39: 1, 40: 0},
    6: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 1, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 1, 27: 1, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 1, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    7: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 1, 15: 1, 16: 1, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 1, 24: 1, 25: 0, 26: 0, 27: 0, 28: 1, 29: 0, 30: 0, 31: 1, 32: 1, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 1, 39: 0, 40: 0},
    8: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 1, 15: 1, 16: 1, 17: 0, 18: 0, 19: 0, 20: 1, 21: 1, 22: 1, 23: 1, 24: 1, 25: 0, 26: 0, 27: 0, 28: 1, 29: 0, 30: 0, 31: 1, 32: 1, 33: 0, 34: 0, 35: 0, 36: 0, 37: 1, 38: 1, 39: 0, 40: 0},
    9: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 1, 18: 1, 19: 1, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 1, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 1},
    10: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 1, 9: 1, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 1, 30: 0, 31: 0, 32: 1, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    11: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1, 22: 1, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 1, 35: 0, 36: 0, 37: 1, 38: 0, 39: 0, 40: 1},
    12: {1: 1, 2: 0, 3: 0, 4: 1, 5: 0, 6: 1, 7: 1, 8: 1, 9: 0, 10: 1, 11: 0, 12: 1, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 1, 21: 1, 22: 1, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 1, 34: 0, 35: 0, 36: 0, 37: 1, 38: 0, 39: 0, 40: 0},
    13: {1: 1, 2: 0, 3: 0, 4: 1, 5: 0, 6: 1, 7: 1, 8: 1, 9: 0, 10: 1, 11: 0, 12: 1, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 1, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    14: {1: 0, 2: 1, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 1, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 1, 19: 1, 20: 0, 21: 0, 22: 0, 23: 1, 24: 1, 25: 0, 26: 1, 27: 1, 28: 0, 29: 1, 30: 1, 31: 0, 32: 0, 33: 0, 34: 1, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    15: {1: 1, 2: 0, 3: 0, 4: 1, 5: 0, 6: 1, 7: 1, 8: 1, 9: 0, 10: 1, 11: 0, 12: 1, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 1, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    16: {1: 1, 2: 0, 3: 0, 4: 1, 5: 0, 6: 1, 7: 1, 8: 1, 9: 0, 10: 2, 11: 0, 12: 1, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 1, 27: 1, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 1, 34: 1, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    17: {1: 0, 2: 0, 3: 1, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 1, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 2, 18: 1, 19: 1, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 1, 26: 0, 27: 0, 28: 1, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 1, 35: 0, 36: 0, 37: 0, 38: 0, 39: 1, 40: 2},
    18: {1: 0, 2: 1, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 1, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 1, 19: 1, 20: 0, 21: 0, 22: 0, 23: 1, 24: 1, 25: 0, 26: 1, 27: 1, 28: 0, 29: 1, 30: 1, 31: 0, 32: 0, 33: 0, 34: 1, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    19: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 1, 15: 1, 16: 1, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 1, 29: 0, 30: 0, 31: 0, 32: 1, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    20: {1: 0, 2: 1, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 1, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 1, 27: 1, 28: 0, 29: 0, 30: 1, 31: 0, 32: 0, 33: 0, 34: 1, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0}
}
NEGATIVE_FAS = {
    1: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 1, 15: 1, 16: 1, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 1, 29: 0, 30: 0, 31: 0, 32: 1, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    2: {1: 0, 2: 0, 3: 1, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 1, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 1, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 1, 26: 0, 27: 0, 28: 1, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 1, 40: 1},
    3: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 1, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 1, 20: 0, 21: 0, 22: 0, 23: 1, 24: 1, 25: 0, 26: 0, 27: 0, 28: 0, 29: 1, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    4: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 1, 18: 1, 19: 1, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 1, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 1},
    5: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 1, 9: 1, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 1, 30: 0, 31: 0, 32: 1, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    6: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 1, 24: 1, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 1, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 1, 39: 0, 40: 0},
    7: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 1, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 1, 27: 1, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 1, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    8: {1: 1, 2: 0, 3: 0, 4: 1, 5: 0, 6: 1, 7: 1, 8: 1, 9: 0, 10: 1, 11: 0, 12: 1, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 1, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    9: {1: 0, 2: 0, 3: 0, 4: 0, 5: 1, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 1, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 1, 36: 1, 37: 0, 38: 0, 39: 0, 40: 0},
    10: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 1, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 1, 20: 0, 21: 0, 22: 0, 23: 1, 24: 1, 25: 0, 26: 0, 27: 0, 28: 0, 29: 1, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    11: {1: 1, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 1, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 1, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 1, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    12: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 1, 9: 1, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 1, 30: 0, 31: 0, 32: 1, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    13: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 1, 19: 0, 20: 0, 21: 0, 22: 0, 23: 1, 24: 1, 25: 0, 26: 0, 27: 0, 28: 0, 29: 1, 30: 0, 31: 1, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 1, 39: 1, 40: 0},
    14: {1: 1, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 1, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 1, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 1, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    15: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 1, 21: 1, 22: 1, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 1, 38: 0, 39: 0, 40: 0},
    16: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 1, 15: 1, 16: 1, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 1, 29: 0, 30: 0, 31: 0, 32: 1, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    17: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 1, 24: 1, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 1, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 1, 39: 0, 40: 0},
    18: {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 1, 19: 0, 20: 0, 21: 0, 22: 0, 23: 1, 24: 1, 25: 0, 26: 0, 27: 0, 28: 0, 29: 1, 30: 0, 31: 1, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 1, 39: 1, 40: 0},
    19: {1: 0, 2: 1, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 1, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 1, 27: 1, 28: 0, 29: 0, 30: 1, 31: 0, 32: 0, 33: 0, 34: 1, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0},
    20: {1: 0, 2: 0, 3: 0, 4: 0, 5: 1, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 1, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 1, 36: 1, 37: 0, 38: 0, 39: 0, 40: 0}
}

def get_ongoing_df_from_file(path):
    columns = ['round', 'arena', 'pirate1', 'pirate2', 'pirate3', 'pirate4', 'fa1', 'fa2', 'fa3', 'fa4', 'opening_odds1', 'opening_odds2', 'opening_odds3', 'opening_odds4', 'closing_odds1', 'closing_odds2', 'closing_odds3', 'closing_odds4']
    df = pd.DataFrame([], columns=columns)
    with open(path) as json_file:
        data = json.load(json_file)
        round = data["round"]
        for arena in range(5):
            pirates = data["pirates"][arena]
            # older rounds don't have fa data
            fas = [0 for pirate in pirates]
            if "foods" in data:
                foods = data["foods"][arena]
                positive_fas = [sum(POSITIVE_FAS[pirate][food] for food in foods) for pirate in pirates]
                negative_fas = [-sum(NEGATIVE_FAS[pirate][food] for food in foods) for pirate in pirates]
                fas = [sum(x) for x in zip(positive_fas, negative_fas)]
            pirate1, pirate2, pirate3, pirate4 = pirates
            fa1, fa2, fa3, fa4 = fas
            _, opening_odds1, opening_odds2, opening_odds3, opening_odds4 = data["openingOdds"][arena]
            _, closing_odds1, closing_odds2, closing_odds3, closing_odds4 = data["currentOdds"][arena]
            df_temp = pd.DataFrame([[round, arena, pirate1, pirate2, pirate3, pirate4, fa1, fa2, fa3, fa4, opening_odds1, opening_odds2, opening_odds3, opening_odds4, closing_odds1, closing_odds2, closing_odds3, closing_odds4]], columns=columns)
            df = pd.concat([df, df_temp], ignore_index=True)
    return df

In [9]:
# backtesting
def get_data_for_round(df, round):
    return df[df['round'] == round]

convert_to_long_format_with_win(get_data_for_round(df, 8641))


,round,arena,win,pirate,fa,pfa,nfa,position,position_factor,is_pos1,is_pos2,is_pos3,is_pos4,opening_odds,closing_odds,match_id,log_opening_implied_winrate
0,8641,0,0,19,2,2,0,1,-0.1821,1,0,0,0,4,4,43205,-1.386294
1,8641,0,0,12,4,5,-1,2,0.0000,0,1,0,0,13,13,43205,-2.564949
2,8641,0,0,2,3,5,-2,3,0.2814,0,0,1,0,13,13,43205,-2.564949
3,8641,0,1,15,3,4,-1,4,0.5523,0,0,0,1,2,2,43205,-0.693147
4,8641,1,0,10,1,1,0,1,-0.1821,1,0,0,0,13,13,43206,-2.564949
5,8641,1,0,20,-1,1,-2,2,0.0000,0,1,0,0,8,12,43206,-2.079442
6,8641,1,0,8,-2,1,-3,3,0.2814,0,0,1,0,13,13,43206,-2.564949
7,8641,1,1,4,3,3,0,4,0.5523,0,0,0,1,2,2,43206,-0.693147
8,8641,2,1,17,4,5,-1,1,-0.1821,1,0,0,0,2,2,43207,-0.693147
9,8641,2,0,7,-1,1,-2,2,0.0000,0,1,0,0,6,6,43207,-1.791759


In [10]:
def generate_bets():
    for i in range(5):
        for j in range(5):
            for k in range(5):
                for l in range(5):
                    for m in range(5):
                        if i != 0 or j != 0 or k != 0 or l != 0 or m != 0:
                            yield (i, j, k, l, m)

len(list(generate_bets()))

3124

In [11]:
def get_odds_of_bet(bet, odds):
    prod = 1
    for arena, pos in enumerate(bet):
        if pos != 0:
            prod *= odds[arena * 4 + pos - 1]
    return prod

tmp = convert_to_long_format_with_win(get_data_for_round(df, 8641))
get_odds_of_bet((4,2,0,0,2), tmp['closing_odds'])

96

In [12]:
def get_estimated_probabilities(df, model):
    return model.predict(df)

# get_estimated_probabilities(tmp, with_position_mnl)

In [13]:
def get_estimated_probability_of_bet(bet, probabilities):
    prod = 1
    for arena, pos in enumerate(bet):
        if pos != 0:
            prod *= probabilities[arena * 4 + pos - 1]
    return prod

# get_estimated_probability_of_bet((4,2,0,0,2), get_estimated_probabilities(tmp, with_position_mnl))

In [14]:
def get_estimated_return_per_unit_of_bet(bet, odds, probabilities):
    return get_odds_of_bet(bet, odds) * get_estimated_probability_of_bet(bet, probabilities)

# get_estimated_return_per_unit_of_bet((4,2,0,0,2), tmp['closing_odds'], get_estimated_probabilities(tmp, with_position_mnl))

In [15]:
def get_max_possible_bet_amount_floor(odds):
    return 1000000 // odds

def get_max_possible_bet_amount_ceil(odds):
    return -(1000000 // -odds)

get_max_possible_bet_amount_floor(96)
get_max_possible_bet_amount_ceil(96)

10417

In [16]:
def get_estimated_return_of_bet_amount(bet, odds, probabilities, bet_amount):
    odds_of_bet = get_odds_of_bet(bet, odds)
    estimated_probability = get_estimated_probability_of_bet(bet, probabilities)
    payout = min(1000000, bet_amount * odds_of_bet)
    return payout * estimated_probability

def get_best_bet_amount_and_estimated_return(bet, odds, probabilities):
    max_possible_bet_amount_floor = get_max_possible_bet_amount_floor(get_odds_of_bet(bet, odds))
    max_possible_bet_amount_ceil = get_max_possible_bet_amount_ceil(get_odds_of_bet(bet, odds))
    estimated_return_bet_amount_floor = get_estimated_return_of_bet_amount(bet, odds, probabilities, max_possible_bet_amount_floor)
    estimated_return_bet_amount_ceil = get_estimated_return_of_bet_amount(bet, odds, probabilities, max_possible_bet_amount_ceil)
    if (estimated_return_bet_amount_floor - max_possible_bet_amount_floor) >= (estimated_return_bet_amount_ceil - max_possible_bet_amount_ceil):
        return max_possible_bet_amount_floor, estimated_return_bet_amount_floor
    else:
        return max_possible_bet_amount_ceil, estimated_return_bet_amount_ceil

def get_best_bet_amount_and_estimated_return_given_personal_cap(bet, odds, probabilities, cap):
    amount, estimated_return = get_best_bet_amount_and_estimated_return(bet, odds, probabilities)
    if cap >= amount:
        return amount, estimated_return
    else:
        return cap, get_estimated_return_of_bet_amount(bet, odds, probabilities, cap)


In [17]:
def get_profitable_bets(odds, probabilities, cap):
    for bet in generate_bets():
        if get_estimated_return_per_unit_of_bet(bet, odds, probabilities) > 1:
            bet_amount, estimated_return = get_best_bet_amount_and_estimated_return_given_personal_cap(bet, odds, probabilities, cap)
            estimated_profit = estimated_return - bet_amount
            return_if_won = min(1000000, bet_amount * get_odds_of_bet(bet, odds))
            yield bet, bet_amount, estimated_return, estimated_profit, return_if_won

# 10 best bets for round 8641
# sorted(list(get_profitable_bets(tmp['closing_odds'], get_estimated_probabilities(tmp, with_position_mnl), 11376)), key=lambda x: x[3], reverse=True)[:10]

In [18]:
def is_winning_bet(bet, winners):
    for arena, pos in enumerate(bet):
        if pos != 0 and winners[arena] != pos:
            return False
    return True

def get_actual_return_of_bet(bet, winners, return_if_won):
    if is_winning_bet(bet, winners):
        return min(1000000, return_if_won)
    else:
        return 0


In [19]:
history_rounds = df["round"].unique().tolist()
len(history_rounds)

3491

In [20]:
from tqdm import tqdm
def backtest_model_with_cap(mnl, cap, history_rounds):
    total_bet_amount = 0
    total_return = 0
    total_potential_bet_amount = cap * 10 * len(history_rounds)
    for round in tqdm(history_rounds):
        tmp = convert_to_long_format_with_win(get_data_for_round(df, round))
        winners = df[df['round'] == round]['winner'].tolist()
        best_ten_bets = sorted(list(get_profitable_bets(tmp['closing_odds'], get_estimated_probabilities(tmp, mnl), cap)), key=lambda x: x[3], reverse=True)[:10]
        for bet, bet_amount, estimated_return, estimated_profit, return_if_won in best_ten_bets:
            total_bet_amount += bet_amount
            total_return += get_actual_return_of_bet(bet, winners, return_if_won)
    return total_bet_amount, total_potential_bet_amount, total_return, total_return - total_bet_amount, 1 + (total_return - total_bet_amount) / total_potential_bet_amount

In [21]:
# next we split positional coefficients for each pirate
per_pirate_fa_and_pos_specification = OrderedDict()
per_pirate_fa_and_pos_names = OrderedDict()
# OK, for asthetic purposes and better-looking P-values
# (Dan is really close to Buck but everyone is much weaker than Gooblah,
#  that's why Buck has a bad-looking P-value) we exlude Gooblah from the intercepts
# You can interpret each exp(intercept) as "how many Goobs each pirate is equivalent to"
pirates_except_goob = [i for i in range(1, 21) if i != 15]
per_pirate_fa_and_pos_specification["intercept"] = pirates_except_goob
per_pirate_fa_and_pos_names["intercept"] = [f'ASC_{i}_{PIRATE_NAMES[i]}' for i in pirates_except_goob]
per_pirate_fa_and_pos_specification["pfa"] = pirates_including_the_first
per_pirate_fa_and_pos_names["pfa"] = [f'PFA_{i}_{PIRATE_NAMES[i]}' for i in pirates_including_the_first]
per_pirate_fa_and_pos_specification["nfa"] = pirates_including_the_first
per_pirate_fa_and_pos_names["nfa"] = [f'NFA_{i}_{PIRATE_NAMES[i]}' for i in pirates_including_the_first]
per_pirate_fa_and_pos_specification["position_factor"] = pirates_including_the_first
per_pirate_fa_and_pos_names["position_factor"] = [f'position_factor_{i}_{PIRATE_NAMES[i]}' for i in pirates_including_the_first]

per_pirate_fa_and_pos_mnl = pl.create_choice_model(data=long_format,
alt_id_col='pirate',
obs_id_col='match_id',
choice_col='win',
specification=per_pirate_fa_and_pos_specification,
model_type='MNL',
names=per_pirate_fa_and_pos_names)
per_pirate_fa_and_pos_mnl.fit_mle(np.zeros(79))
per_pirate_fa_and_pos_mnl.get_statsmodels_summary()

Log-likelihood at zero: -24,197.7681
Initial Log-likelihood: -24,197.7681


/home/arsdragonfly/neofoodclub/envs/default/lib/python3.10/site-packages/scipy/optimize/_minimize.py:555: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


Estimation Time for Point Estimation: 9.20 seconds.
Final log-likelihood: -18,727.7708


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                         win   No. Observations:               17,455
Model:             Multinomial Logit Model   Df Residuals:                   17,376
Method:                                MLE   Df Model:                           79
Date:                     Fri, 05 May 2023   Pseudo R-squ.:                   0.226
Time:                             18:45:08   Pseudo R-bar-squ.:               0.223
AIC:                            37,613.542   Log-Likelihood:            -18,727.771
BIC:                            38,227.165   LL-Null:                   -24,197.768
==================================================================================================
                                     coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
ASC_1_Dan                         -0.5970      0.128     -4.654      0.000      -0.848      -0.346
ASC_2_Sproggie                    -2.4092      0.173    -13.894      0.000      -2.749      -2.069
ASC_3_Orvinn                      -3.3697      0.209    -16.149      0.000      -3.779      -2.961
ASC_4_Lucky                       -1.2734      0.128     -9.943      0.000      -1.524      -1.022
ASC_5_Edmund                      -1.7957      0.131    -13.738      0.000      -2.052      -1.539
ASC_6_Peg Leg                     -2.4360      0.141    -17.317      0.000      -2.712      -2.160
ASC_7_Bonnie                      -2.1911      0.147    -14.874      0.000      -2.480      -1.902
ASC_8_Puffo                       -2.9663      0.208    -14.278      0.000      -3.373      -2.559
ASC_9_Stuff                       -3.8522      0.200    -19.254      0.000      -4.244      -3.460
ASC_10_Squire                     -3.2861      0.177    -18.539      0.000      -3.633      -2.939
ASC_11_Crossblades                -2.8946      0.173    -16.750      0.000      -3.233      -2.556
ASC_12_Stripey                    -2.1055      0.147    -14.309      0.000      -2.394      -1.817
ASC_13_Ned                        -1.5616      0.146    -10.675      0.000      -1.848      -1.275
ASC_14_Fairfax                    -2.5035      0.160    -15.629      0.000      -2.817      -2.190
ASC_16_Franchisco                 -1.2911      0.136     -9.462      0.000      -1.559      -1.024
ASC_17_Federismo                  -1.1330      0.134     -8.454      0.000      -1.396      -0.870
ASC_18_Blackbeard                 -2.0413      0.152    -13.470      0.000      -2.338      -1.744
ASC_19_Buck                       -0.4960      0.127     -3.904      0.000      -0.745      -0.247
ASC_20_Tailhook                   -1.5589      0.130    -11.966      0.000      -1.814      -1.304
PFA_1_Dan                          0.1574      0.022      7.086      0.000       0.114       0.201
PFA_2_Sproggie                     0.2632      0.035      7.550      0.000       0.195       0.332
PFA_3_Orvinn                       0.2519      0.041      6.202      0.000       0.172       0.332
PFA_4_Lucky                        0.1610      0.040      4.038      0.000       0.083       0.239
PFA_5_Edmund                       0.2769      0.039      7.172      0.000       0.201       0.353
PFA_6_Peg Leg                      0.3155      0.060      5.276      0.000       0.198       0.433
PFA_7_Bonnie                       0.2462      0.036      6.813      0.000       0.175       0.317
PFA_8_Puffo                        0.2848      0.039      7.295      0.000       0.208       0.361
PFA_9_Stuff                        0.4108      0.076      5.403      0.000       0.262       0.560
PFA_10_Squire                      0.1994      0.082      2.420      0.016       0.038       0.361
PFA_11_Crossblades          

In [22]:
def get_current_round_df_from_file(path):
    columns = ['round', 'arena', 'pirate1', 'pirate2', 'pirate3', 'pirate4', 'fa1', 'fa2', 'fa3', 'fa4', 'pfa1', 'pfa2', 'pfa3', 'pfa4', 'nfa1', 'nfa2', 'nfa3', 'nfa4', 'opening_odds1', 'opening_odds2', 'opening_odds3', 'opening_odds4', 'closing_odds1', 'closing_odds2', 'closing_odds3', 'closing_odds4']
    df = pd.DataFrame([], columns=columns)
    with open(path) as json_file:
        data = json.load(json_file)
        round = data["round"]
        for arena in range(5):
            pirates = data["pirates"][arena]
            # older rounds don't have fa data
            fas = [0 for pirate in pirates]
            positive_fas = [0 for pirate in pirates]
            negative_fas = [0 for pirate in pirates]
            if "foods" in data:
                foods = data["foods"][arena]
                positive_fas = [sum(POSITIVE_FAS[pirate][food] for food in foods) for pirate in pirates]
                negative_fas = [-sum(NEGATIVE_FAS[pirate][food] for food in foods) for pirate in pirates]
                fas = [sum(x) for x in zip(positive_fas, negative_fas)]
            pirate1, pirate2, pirate3, pirate4 = pirates
            fa1, fa2, fa3, fa4 = fas
            pfa1, pfa2, pfa3, pfa4 = positive_fas
            nfa1, nfa2, nfa3, nfa4 = negative_fas
            _, opening_odds1, opening_odds2, opening_odds3, opening_odds4 = data["openingOdds"][arena]
            _, closing_odds1, closing_odds2, closing_odds3, closing_odds4 = data["currentOdds"][arena]
            df_temp = pd.DataFrame([[round, arena, pirate1, pirate2, pirate3, pirate4, fa1, fa2, fa3, fa4, pfa1, pfa2, pfa3, pfa4, nfa1, nfa2, nfa3, nfa4, opening_odds1, opening_odds2, opening_odds3, opening_odds4, closing_odds1, closing_odds2, closing_odds3, closing_odds4]], columns=columns)
            df = pd.concat([df, df_temp], ignore_index=True)
    return df

In [23]:
def get_round(round_number):
    return convert_to_long_format_without_win(convert_df(get_current_round_df_from_file(f"raw_json/{round_number}.json")))
round_8643 = get_round(8643)
round_8643

,round,arena,pirate,fa,pfa,nfa,position,position_factor,is_pos1,is_pos2,is_pos3,is_pos4,opening_odds,closing_odds,match_id,log_opening_implied_winrate
0,8643,0,2,-1,2,-3,1,-0.1821,1,0,0,0,13,13,43215,-2.564949
1,8643,0,16,2,2,0,2,0.0000,0,1,0,0,2,2,43215,-0.693147
2,8643,0,3,0,1,-1,3,0.2814,0,0,1,0,13,13,43215,-2.564949
3,8643,0,5,2,2,0,4,0.5523,0,0,0,1,2,2,43215,-0.693147
4,8643,1,9,-1,0,-1,1,-0.1821,1,0,0,0,13,13,43216,-2.564949
5,8643,1,18,1,2,-1,2,0.0000,0,1,0,0,13,13,43216,-2.564949
6,8643,1,1,4,4,0,3,0.2814,0,0,1,0,2,2,43216,-0.693147
7,8643,1,8,-1,2,-3,4,0.5523,0,0,0,1,13,13,43216,-2.564949
8,8643,2,19,0,1,-1,1,-0.1821,1,0,0,0,2,2,43217,-0.693147
9,8643,2,13,2,3,-1,2,0.0000,0,1,0,0,2,2,43217,-0.693147


In [24]:
per_pirate_fa_and_pos_mnl.predict(round_8643)

array([0.0524989 , 0.38679778, 0.04288192, 0.5178214 , 0.0093473 ,
       0.08036646, 0.87018676, 0.04009948, 0.50777671, 0.29919012,
       0.05515694, 0.13787622, 0.08246714, 0.33481426, 0.39802827,
       0.18469032, 0.04334512, 0.86464406, 0.05110225, 0.04090857])

In [25]:
import string
def parse_bets(s):
    indices = [string.ascii_lowercase.find(c) for c in s]
    values = []
    for index in indices:
        values.append(index // 5)
        values.append(index % 5)

In [26]:
def get_best_10_bets(round, odds, model, cap):
    return sorted(list(get_profitable_bets(
        odds,
        get_estimated_probabilities(get_round(round), model), cap)),
        key=lambda x: x[3], reverse=True)[:10]

In [27]:
def get_bet_url(round, bets):
    bets_array = [v for bet in bets for v in bet[0]]
    # turn bets_array into chunks of 2 (pad 0 if necessary)
    if len(bets_array) % 2 == 1:
        bets_array.append(0)
    bets_chunks = [bets_array[i:i+2] for i in range(0, len(bets_array), 2)]
    # turn bets_chunks into a string
    bets_string = "".join([string.ascii_lowercase[5 * x + y] for x, y in bets_chunks])
    return f"/#round={round}&b={bets_string}"

In [28]:
from coloraide import Color, stop
def get_background_for_payout(payout):
    return Color.interpolate(['#50c17f', stop('transparent', np.exp(-1)), '#f76c6c'], space='srgb', premultiplied=True)(np.exp(-payout)).to_string()

get_background_for_payout(1.2)

'rgb(80 193 127 / 0.18127)'

In [29]:
def get_payouts(round, model, odds_override=None):
    round_df = get_round(round)
    estimated_probabilities = get_estimated_probabilities(round_df, model)
    estimated_probabilities_series = pd.Series(estimated_probabilities, name="estimated_probabilities")
    round_df = round_df.join(estimated_probabilities_series)
    if odds_override is not None:
        round_df["custom_odds"] = pd.Series(odds_override, name="custom_odds")
    else:
        round_df["custom_odds"] = round_df["closing_odds"]
    round_df["payouts"] = round_df.apply(lambda row: row["custom_odds"] * row["estimated_probabilities"], axis=1)
    return round_df.style.applymap(lambda x: f"background-color: {get_background_for_payout(x)}", subset=["payouts"])

In [30]:
current_round = 8764

In [31]:
manual_odds = get_round(current_round)['closing_odds'].tolist()
manual_odds

[4, 7, 2, 7, 2, 13, 2, 5, 7, 3, 7, 2, 2, 11, 4, 6, 13, 2, 3, 5]

In [32]:
per_pirate_fa_and_pos_mnl.predict(get_round(current_round))

array([0.20054977, 0.14587641, 0.49595898, 0.15761484, 0.31331474,
       0.03083773, 0.47784937, 0.17799816, 0.09552547, 0.24521051,
       0.16318461, 0.49607942, 0.51634561, 0.11363566, 0.1909996 ,
       0.17901913, 0.06744558, 0.52990105, 0.26575824, 0.13689512])

In [33]:
# manual_odds = [5, 2, 13, 6, 13, 13, 2, 2, 6, 13, 4, 2, 4, 3, 3, 3, 2, 4, 9, 3]

In [33]:
get_payouts(current_round, per_pirate_fa_and_pos_mnl, odds_override=manual_odds)

,round,arena,pirate,fa,pfa,nfa,position,position_factor,is_pos1,is_pos2,is_pos3,is_pos4,opening_odds,closing_odds,match_id,log_opening_implied_winrate,intercept,estimated_probabilities,custom_odds,payouts
0,8764,0,3,5,5,0,1,-0.182100,1,0,0,0,4,4,43820,-1.386294,1.000000,0.200550,4,0.802199
1,8764,0,4,-1,2,-3,2,0.000000,0,1,0,0,5,7,43820,-1.609438,1.000000,0.145876,7,1.021135
2,8764,0,19,-3,0,-3,3,0.281400,0,0,1,0,2,2,43820,-0.693147,1.000000,0.495959,2,0.991918
3,8764,0,8,0,2,-2,4,0.552300,0,0,0,1,5,7,43820,-1.609438,1.000000,0.157615,7,1.103304
4,8764,1,15,2,4,-2,1,-0.182100,1,0,0,0,3,2,43821,-1.098612,1.000000,0.313315,2,0.626629
5,8764,1,14,1,2,-1,2,0.000000,0,1,0,0,13,13,43821,-2.564949,1.000000,0.030838,13,0.400890
6,8764,1,1,5,5,0,3,0.281400,0,0,1,0,2,2,43821,-0.693147,1.000000,0.477849,2,0.955699
7,8764,1,20,1,1,0,4,0.552300,0,0,0,1,6,5,43821,-1.791759,1.000000,0.177998,5,0.889991
8,8764,2,5,-2,1,-3,1,-0.182100,1,0,0,0,8,7,43822,-2.079442,1.000000,0.095525,7,0.668678
9,8764,2,2,2,3,-1,2,0.000000,0,1,0,0,3,3,43822,-1.098612,1.000000,0.245211,3,0.735632


In [34]:
get_best_10_bets(current_round, manual_odds, per_pirate_fa_and_pos_mnl, 1)

[((4, 0, 3, 2, 2), 1, 1.6695694491816786, 0.6695694491816786, 1078),
 ((4, 3, 3, 2, 2), 1, 1.595605424522293, 0.595605424522293, 2156),
 ((4, 0, 3, 2, 0), 1, 1.5753596301183825, 0.5753596301183825, 539),
 ((2, 0, 3, 2, 2), 1, 1.5452275672778213, 0.5452275672778213, 1078),
 ((0, 0, 3, 2, 2), 1, 1.5132453559226413, 0.5132453559226413, 154),
 ((4, 3, 3, 2, 0), 1, 1.505569218832055, 0.5055692188320551, 1078),
 ((3, 0, 3, 2, 2), 1, 1.501015258864763, 0.501015258864763, 308),
 ((4, 4, 3, 2, 2), 1, 1.48590142287881, 0.4859014228788101, 5390),
 ((2, 3, 3, 2, 2), 1, 1.4767720442407184, 0.4767720442407184, 2156),
 ((4, 0, 0, 2, 2), 1, 1.461595691028219, 0.46159569102821907, 154)]

In [35]:
get_bet_url(current_round, get_best_10_bets(current_round, manual_odds, per_pirate_fa_and_pos_mnl, 1))

'/#round=8764&b=urosmurcdmaroskproxmnroam'

In [36]:
get_best_10_bets(current_round, manual_odds, per_pirate_fa_and_pos_mnl, 11424)

[((0, 0, 3, 2, 0), 11424, 16311.833025283127, 4887.833025283127, 879648),
 ((4, 0, 0, 2, 0), 11424, 15755.082128099, 4331.082128099, 879648),
 ((0, 0, 0, 2, 2), 11424, 15133.880802019807, 3709.8808020198067, 251328),
 ((0, 0, 4, 2, 2), 11424, 15015.213470364313, 3591.2134703643133, 502656),
 ((3, 0, 0, 2, 2), 11424, 15011.568296419411, 3587.568296419411, 502656),
 ((3, 0, 4, 2, 2), 11363, 14814.332248841663, 3451.3322488416634, 999944),
 ((0, 0, 3, 4, 2), 11424, 14854.940118854145, 3430.940118854145, 959616),
 ((4, 0, 3, 0, 2), 10204, 13629.113287725982, 3425.1132877259824, 999992),
 ((0, 0, 3, 2, 2), 6493, 9825.50209600571, 3332.50209600571, 999922),
 ((2, 0, 0, 2, 0), 11424, 14581.715807627663, 3157.715807627663, 879648)]

In [37]:
get_bet_url(current_round, get_best_10_bets(current_round, manual_odds, per_pirate_fa_and_pos_mnl, 11424))

'/#round=8764&b=areakackempcnematodcarmak'